In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day'
]

In [ ]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'day': 4, 'sunny': 5, 'rainy': 6}


In [ ]:
sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?'
]

In [ ]:
tokenizer = Tokenizer(num_words = 100)
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
print(word_index)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}


Sequence

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print(word_index)
print(sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 4, 5], [1, 2, 3, 6, 5], [2, 7, 4, 1]]


ทดสอบกับคำที่ไม่เคยเห็น

In [ ]:
test_data = [
  'Today is a snowy day',
  'Will it be rainy tomorrow?'
]


test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'today': 1, 'is': 2, 'a': 3, 'sunny': 4, 'day': 5, 'rainy': 6, 'it': 7}
[[1, 2, 3, 5], [7, 6]]


In [ ]:
# re-tokenize with an OOV token
tokenizer = Tokenizer(num_words = 100, oov_token="")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

test_sequences = tokenizer.texts_to_sequences(test_data)
print(word_index)
print(test_sequences)

{'': 1, 'today': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, 'rainy': 7, 'it': 8}
[[2, 3, 4, 1, 6], [1, 8, 1, 7, 1]]


ทดสอบเรื่อง Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


sentences = [
    'Today is a sunny day',
    'Today is a rainy day',
    'Is it sunny today?',
    'I really enjoyed walking in the snow today'
]


# Re-tokenize with the new sentences from above
tokenizer = Tokenizer(num_words = 100, oov_token="")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
print(sequences)

[[2, 3, 4, 5, 6], [2, 3, 4, 7, 6], [3, 8, 5, 2], [9, 10, 11, 12, 13, 14, 15, 2]]


In [ ]:
padded = pad_sequences(sequences)

print(padded)

[[ 0  0  0  2  3  4  5  6]
 [ 0  0  0  2  3  4  7  6]
 [ 0  0  0  0  3  8  5  2]
 [ 9 10 11 12 13 14 15  2]]


In [ ]:
padded = pad_sequences(sequences, padding='post')

print(padded)


[[ 2  3  4  5  6  0  0  0]
 [ 2  3  4  7  6  0  0  0]
 [ 3  8  5  2  0  0  0  0]
 [ 9 10 11 12 13 14 15  2]]


In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=6)

print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [11 12 13 14 15  2]]


In [ ]:
padded = pad_sequences(sequences, padding='post', maxlen=6, truncating='post')

print(padded)

[[ 2  3  4  5  6  0]
 [ 2  3  4  7  6  0]
 [ 3  8  5  2  0  0]
 [ 9 10 11 12 13 14]]


Handle StopWord and Data Preprocessing

In [ ]:
!wget --no-check-certificate --no-cache \
    https://storage.googleapis.com/learning-datasets/binary-emotion.csv \
    -O /tmp/binary-emotion.csv

--2026-01-05 15:53:52--  https://storage.googleapis.com/learning-datasets/binary-emotion.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.144.207, 74.125.200.207, 74.125.130.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.144.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2690504 (2.6M) [text/csv]
Saving to: ‘/tmp/binary-emotion.csv’

/tmp/binary-emotion 100%[===================>]   2.57M  2.10MB/s    in 1.2s    

2026-01-05 15:53:54 (2.10 MB/s) - ‘/tmp/binary-emotion.csv’ saved [2690504/2690504]



In [ ]:
from bs4 import BeautifulSoup
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [ ]:
import csv
sentences=[]
labels=[]
with open('/tmp/binary-emotion.csv', encoding='UTF-8') as csvfile:
  reader = csv.reader(csvfile, delimiter=",")
  for row in reader:
    labels.append(int(row[0]))
    sentence = row[1].lower()
    sentence = sentence.replace(",", " , ")
    sentence = sentence.replace(".", " . ")
    sentence = sentence.replace("-", " - ")
    sentence = sentence.replace("/", " / ")
    soup = BeautifulSoup(sentence)
    sentence = soup.get_text()
    words = sentence.split()
    filtered_sentence = ""
    for word in words:
        word = word.translate(table)
        if word not in stopwords:
            filtered_sentence = filtered_sentence + word + " "
    sentences.append(filtered_sentence)

print(len(labels))
print(len(sentences))



35327
35327


In [ ]:
print(sentences[0:10])
print(labels[0:10])

['know listenin bad habit earlier started freakin part  ', 'layin n bed headache ughhhh waitin call ', 'wants hang friends soon ', 'want trade someone houston tickets no one will ', 're  pinging didnt go prom bc bf didnt like friends ', 'sleep not thinking old friend want married now damn  wants 2 scandalous ', 'hmmm djhero com  ', 'charlene love miss ', 'sorry least friday ', 'ugh beat stupid song get next rude ']


[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]

In [ ]:
training_size = 28000

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
vocab_size = 20000
embedding_dim = 32
max_length = 10
trunc_type='post'
padding_type='post'
oov_tok = ""

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length,
                               padding=padding_type, truncating=trunc_type)

In [ ]:
print(training_sequences[0])
print(training_padded[0])


print(word_index)

[18, 3257, 47, 4770, 613, 508, 951, 423]
[  18 3257   47 4770  613  508  951  423    0    0]
{'': 1, 'just': 2, 'not': 3, 'now': 4, 'day': 5, 'get': 6, 'no': 7, 'good': 8, 'like': 9, 'go': 10, 'dont': 11, 'work': 12, 'today': 13, 'got': 14, 'cant': 15, 'going': 16, 'one': 17, 'know': 18, 'back': 19, 'time': 20, 'love': 21, 'really': 22, 'will': 23, 'lol': 24, 'com': 25, 'u': 26, 'see': 27, 'want': 28, 'can': 29, 'home': 30, 'new': 31, 'think': 32, '2': 33, 'still': 34, 'oh': 35, 'miss': 36, 'much': 37, 'off': 38, 'need': 39, 'night': 40, 'last': 41, 'feel': 42, 'sad': 43, 'wish': 44, 'hope': 45, 'sorry': 46, 'bad': 47, 'didnt': 48, 'great': 49, 'morning': 50, 'thanks': 51, 'haha': 52, 'tomorrow': 53, 'twitter': 54, 'happy': 55, '3': 56, 'right': 57, 'tonight': 58, 'though': 59, 'gonna': 60, 'week': 61, 'fun': 62, 'make': 63, 'better': 64, 'weekend': 65, 'getting': 66, 'way': 67, 'nice': 68, 'come': 69, 'school': 70, 'sleep': 71, 'yeah': 72, 'hate': 73, 'people': 74, 'next': 75, 'days':